In [ ]:
!pip install dnspython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install python-whois

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
import dns.resolver
import whois
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from scipy.sparse import hstack
import joblib
from tqdm import tqdm

### **데이터 전처리**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

malicious_path = "/content/drive/MyDrive/gra_project/졸업프로젝트/[졸프]데이터_csv/malicous.csv"
졸업프로젝트/[졸프]데이터_csv/benign.csv"

benign_data = pd.read_csv(benign_path)
malicious_data = pd.read_csv(malicious_path)

benign_data = benign_data.iloc[:, [0]]
malicious_data = malicious_data.iloc[:, [0]]


benign_data.columns=['url']
malicious_data.columns=['url']

benign_data['label'] = 0
malicious_data['label'] = 1

# url 열에서 중복되는 값 제거 (첫 번째만 남기고 나머지 제거)
benign_data_clean = benign_data.drop_duplicates(subset=['url'])
malicious_data_clean = malicious_data.drop_duplicates(subset=['url'])


#data=pd.concat([benign_data,malicious_data],ignore_index=True)




In [ ]:
print(len(benign_data_clean))
print(len(malicious_data_clean))
print(len(malicious_data_clean)+len(benign_data_clean))

149707
94067
243774


In [ ]:
# URL의 길이
def length_url(url):
    return len(url)

In [ ]:
### 특수 문자 개수 카운트
# 특수 문자 리스트
special_chars = ['@', '?', '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '/']

# 특수 문자의 개수를 특징으로 추가
def count_special_characters(url):
    return sum(url.count(char) for char in special_chars)

In [ ]:
# HTTPS 존재 여부
def https_exist(url):
    return 1 if url.startswith('https://') else 0
# df['https'] = df['url'].apply(lambda x: 1 if x.startswith('https://') else 0)

In [ ]:
import dns.resolver
from urllib.parse import urlparse

def get_ttl(domain):
    try:
        # URL에서 도메인 이름만 추출
        parsed_url = urlparse(domain)
        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        # A 레코드로 TTL 정보 조회
        answers = dns.resolver.resolve(domain_name, 'A')

        # 첫 번째 A 레코드의 TTL 값 반환
        ttl = answers.ttl
        return ttl
    except Exception as e:
        print(f"Error resolving TTL for {domain_name}: {e}")
        return None

In [ ]:
# name server 가져오기
def get_ns(domain):
    try:

      # URL에서 도메인 이름만 추출
        parsed_url = urlparse(domain)
        domain_name = parsed_url.netloc if parsed_url.netloc else parsed_url.path

        answers = dns.resolver.resolve(domain_name, 'NS')
        ns_records = [str(rdata.target).rstrip('.') for rdata in answers]
        return ns_records
    except Exception as e:
        #print(f"Error resolving NS for {domain_name}: {e}")
        return []

In [ ]:
# IP 혹은 이메일이 URL에 포함되어 있는 경우 악성으로 판단
def ip_email_exist(url):
    # IP 주소 패턴 (IPv4)
    ip_pattern = r'(?:\d{1,3}\.){3}\d{1,3}'

    # 이메일 주소 패턴
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'

    # IP 주소가 포함된 경우
    if re.search(ip_pattern, url):
        return 1

    # 이메일 주소가 포함된 경우
    if re.search(email_pattern, url):
        return 1

    # 둘 다 해당되지 않으면 피싱 URL 아님
    return 0

In [ ]:
# 단축 url
def is_short_url(url):
    # 단축 URL 패턴 (여기서는 일부만 포함, 필요한 만큼 추가 가능)
    short_url_pattern = r'bit\.ly|goo\.gl|t\.co|tinyurl\.com|me2\.kr|is\.gd|shrtco\.de|ow\.ly|buff\.ly|mcaf\.ee|surl\.kr'

    # 단축 URL 도메인이 포함되어 있는지 확인
    if re.search(short_url_pattern, url):
        return 1  # 단축 URL이 포함된 경우
    else:
        return 0  # 포함되지 않은 경우


In [ ]:
def file_extension(url):
    # 파일 확장자 패턴
    file_extension_pattern = r'\.apk|\.pdf|\.jpg|\.png|\.zip|\.html|\.docx|\.xlsx|\.pptx'

    # URL에 파일 확장자가 포함되어 있는지 확인
    if re.search(file_extension_pattern, url, re.IGNORECASE):
        return 1  # 확장자 포함
    else:
        return 0  # 확장자 불포함

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_domains(url, name_servers):
    # URL 벡터화
    vectorizer = TfidfVectorizer()
    # 네임 서버를 하나의 문자열로 변환
    name_servers_str = ' '.join(name_servers) if name_servers else ''
    # 두 도메인을 포함한 리스트
    domains = [url, name_servers_str]

    # TF-IDF 행렬 생성
    tfidf_matrix = vectorizer.fit_transform(domains)

    # 코사인 유사도 계산
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

    return similarity[0][0]

def ns_domain_match_cosine(row):
    if not row['name_server']:
        return None
    url = row['url']
    name_servers = row['name_server']

    # 코사인 유사도 계산
    similarity_score = cosine_similarity_domains(url, name_servers)

    # 유사도가 0.5 이상이면 일치로 판단 (임의 기준)
    return 1 if similarity_score >= 0.5 else 0



In [ ]:
# tqdm을 pandas에 확장
tqdm.pandas()

### benign_data

In [ ]:
# 'character_count' 열 추가
benign_data_clean['url_length'] = benign_data_clean['url'].apply(length_url)
benign_data_clean['character_count'] = benign_data_clean['url'].apply(count_special_characters)
benign_data_clean['https'] = benign_data_clean['url'].apply(https_exist)

In [ ]:
# get_ttl 및 get_ns 함수를 benign_data_clean['url']에 적용하면서 진행 상황을 표시
benign_data_clean['ttl'] = benign_data_clean['url'].progress_apply(get_ttl)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
benign_data_clean['name_server'] = benign_data_clean['url'].progress_apply(get_ns)

100%|██████████| 30000/30000 [53:34<00:00,  9.33it/s]


In [ ]:
benign_data_clean['IP/email'] = benign_data_clean['url'].apply(ip_email_exist)
benign_data_clean['short_url'] = benign_data_clean['url'].apply(is_short_url)
benign_data_clean['file_extension'] = benign_data_clean['url'].apply(file_extension)

In [ ]:
# 새로운 열 추가
benign_data_clean['ns_domain_match'] = benign_data_clean.progress_apply(ns_domain_match_cosine, axis=1)

100%|██████████| 13465/13465 [00:36<00:00, 371.31it/s]


In [ ]:
benign_data_clean

In [ ]:
#수집한 데이터 저장
benign_data_clean.to_csv('benign_data_1015.csv', index=False, encoding='utf-8')

### malicious_data

In [ ]:
# 'character_count' 열 추가
malicious_data_clean['url_length'] = malicious_data_clean['url'].apply(length_url)
malicious_data_clean['character_count'] = malicious_data_clean['url'].apply(count_special_characters)
malicious_data_clean['https'] = malicious_data_clean['url'].apply(https_exist)

In [ ]:
# get_ttl 및 get_ns 함수를 malicious_data_clean['url']에 적용하면서 진행 상황을 표시
malicious_data_clean['ttl'] = malicious_data_clean['url'].progress_apply(get_ttl)


In [ ]:
malicious_data_clean['name_server'] = malicious_data_clean['url'].progress_apply(get_ns)

In [ ]:
malicious_data_clean['IP/email'] = malicious_data_clean['url'].apply(ip_email_exist)
malicious_data_clean['short_url'] = malicious_data_clean['url'].apply(is_short_url)
malicious_data_clean['file_extension'] = malicious_data_clean['url'].apply(file_extension)

In [ ]:
# 새로운 열 추가
malicious_data_clean['ns_domain_match'] = malicious_data_clean.progress_apply(ns_domain_match_cosine, axis=1)

In [ ]:
malicious_data_clean

In [ ]:
#수집한 데이터 저장
malicious_data_clean.to_csv('malicious_data_1015.csv', index=False, encoding='utf-8')

NameError: name 'malicious_data' is not defined